In [1]:
import tensorflow as tf
import os
import random
import math
import sys

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [11]:
#验证集数量
_NUM_TEST = 5
#随机种子
_RANDOM_SEED = 0
#数据块
_NUM_SHARDS = 2
#数据集路径
DATASET_DIR = "pic/"
#标签文件名字
LABELS_FILENAME = "labels.txt"

#定义tfrecord文件的路径和名字
def _get_dataset_filename(dataset_dir, split_name, shard_id):
    output_filename = 'image_%s_%05d-of%05d.tfrecord' % (split_name,shard_id,_NUM_SHARDS)
    return os.path.join(dataset_dir,output_filename)

#判断tfrecord文件是否存在
def _dataset_exists(dataset_dir):
    for split_name in ['train','test']:
        for shard_id in range(_NUM_SHARDS):
            #定义tfrecord文件的路径和名字
            output_filename = _get_dataset_filename(dataset_dir, split_name, shard_id)
        if not tf.gfile.Exists(output_filename):
            return False
    return True
    
#获取所有文件以及分类
def _get_filename_and_classes(dataset_dir):
    #数据目录
    directories = []
    #分类名称
    class_names = []
    
    for filename in os.listdir(dataset_dir):
        path = os.path.join(dataset_dir, filename)
        
        if os.path.isdir(path):
            directories.append(path)
            class_names.append(filename)
    photo_filenames = []
    #循环每个分类文件夹
    for directory in directories:
        for filename in os.listdir(directory):
            path = os.path.join(directory,filename)
            #把图片加入图片列表
            photo_filenames.append(path)
            
    return photo_filenames, class_names


def int64_feature(values):
    if not isinstance(values, (tuple, list)):
        values = [values]
    return tf.train.Feature(int64_list=tf.train.Int64List(value=values))

def bytes_feature(values):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[values]))

def image_to_tfexample(image_data, image_format, class_id):
    return tf.train.Example(features=tf.train.Features(feature={
        'image/encoded':bytes_feature(image_data),
        'image/format':bytes_feature(image_format),
        'image/class/label':int64_feature(class_id),
    }))

def write_label_file(labels_to_class_names, dataset_dir, filename=LABELS_FILENAME):
    labels_filename = os.path.join(dataset_dir, filename)
    with tf.gfile.Open(labels_filename, 'w') as f:
        for label in labels_to_class_names:
            class_name = labels_to_class_names[label]
            print(label,class_name)
            f.write('%d:%s\n' % (label, class_name))
            
#把数据转为TFRecord格式
def _convert_dataset(split_name, filenames, class_names_to_ids, dataset_dir):
    assert split_name in ['train','test']
    num_per_shard = int(len(filenames)/ _NUM_SHARDS)
    with tf.Graph().as_default():
        with tf.Session() as sess:
            for shard_id in range(_NUM_SHARDS):
                output_filename = _get_dataset_filename(dataset_dir, split_name, shard_id)
                with tf.python_io.TFRecordWriter(output_filename) as tfrecord_writer:
                    #块开始
                    start_ndx = shard_id * num_per_shard
                    #块结束
                    end_ndx = min((shard_id+1)*num_per_shard, len(filenames))
                    
                    for i in range(start_ndx, end_ndx):
                        try:
                            sys.stdout.write('\r>>Converting image %d%d shard %d' % (i+1, len(filenames), shard_id))
                            sys.stdout.flush()
                            #读取图片
                            image_data = tf.gfile.FastGFile(filenames[i],'rb').read()
                            #获取图片类别
                            class_name = os.path.basename(os.path.dirname(filenames[i]))
                            #找到图片对于ID
                            class_id = class_names_to_ids[class_name]
                            #生产tfrecord文件
                            example = image_to_tfexample(image_data, b'jpg', class_id)
                            tfrecord_writer.write(example.SerializeToString())
                        except IOError as e:
                            print("Could not read:",filenames[i])
                            print("Error:",e)
                            print("Skip it\n")
    sys.stdout.write('\n')
    sys.stdout.flush()
    
if __name__ == '__main__':
    #判断是否tfrecord文件存在
    if _dataset_exists(DATASET_DIR):
        print('tfrecord文件已存在')
    else:
        #获取所有文件和分类
        photo_filenames, class_names = _get_filename_and_classes(DATASET_DIR)
        #将分类改为字典
        class_names_to_ids = dict(zip(class_names,range(len(class_names))))
        
        #进数据切分训练集和测试集
        random.seed(_RANDOM_SEED)
        random.shuffle(photo_filenames)
        training_filenames = photo_filenames[_NUM_TEST:]
        testing_filenames = photo_filenames[:_NUM_TEST]
        
        #数据转换
        _convert_dataset('train', training_filenames, class_names_to_ids, DATASET_DIR)
        _convert_dataset('test', testing_filenames, class_names_to_ids, DATASET_DIR)
        
        #输出label文件
        labels_to_class_names = dict(zip(range(len(class_names)), class_names))
        write_label_file(labels_to_class_names, DATASET_DIR)

>>Converting image 5455 shard 1
>>Converting image 45 shard 1
0 plane
1 dog
2 car
